Берем результаты других ноутбуков и объединяем их тут

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join, isdir

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

In [3]:
datename = 'beijing'

In [4]:
path = f'./eval/total/'
dirs = [join(path, f) for f in listdir(path) if isdir(join(path, f))]

In [6]:
results = []
for d in dirs:
    if datename in d:
        res = [pd.read_csv(join(d, f), header=[0, 1], skipinitialspace=True) for f in listdir(d) if isfile(join(d, f)) and 'csv' in f]
        results.extend(res)

results = pd.concat(results)
results = results.rename(columns={'Unnamed: 0_level_0':'', 
                                  'Unnamed: 1_level_0':'', 
                                  'Unnamed: 2_level_0':''})

In [7]:
import numpy as np
import json

with open('/workspace/TabDDPM_copy/TabDDPM/Model_names_simple.json', 'r') as openfile:
    naming = json.load(openfile)

def get_positions(s, substring):
    positions = []
    start = 0
    while True:
        start = s.find(substring, start)
        if start == -1:
            break
        positions.append(start)
        start += len(substring)
    return positions
    
def get_simple_name(model_name):
    try:
        pos = get_positions(model_name, '_')[-2]
        model_name_short = model_name[pos+1:]

        if naming.get(model_name_short):
            return naming[model_name_short]
        else:
            pos = get_positions(model_name, '_')[-3]
            model_name_short = model_name[pos+1:]
            return naming[model_name_short]
    except:
        return 'TabDDPM'

results[('', 'Model name')] = results[('', 'Type')].apply(lambda x: get_simple_name(x))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = results[[
    ('', 'Model name') ,
    ('similarity', 'Column Shapes Score, %') ,
    ('similarity', 'Column Pair Trends Score, %') ,
    ('similarity', 'Overall Score (Average), %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation, %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation std, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score std, %') ,
    ('mle', 'RMSE обучения на синтетических данных') ,
    ('mle', 'RMSE обучения на синтетических данных, std') ,
    ('detection', 'Score') ,
    ('DCR', 'Score') ,
    ('quality', 'alpha precision') ,
    ('quality', 'beta recall') ,
]].sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

df

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
0      TabDDPM              99.050274                   97.467421   
1   Constant 1              97.989454                   96.696962   
2   Constant 5              98.079038                   96.579441   
3      Steps 5              97.683892                   96.690476   
4     Cosine 3              97.649300                   96.664884   
5     Linear 2              97.773698                   96.299449   
6   Constant 4              97.749749                   96.175113   
7     Cosine 2              97.666817                   96.106487   
8   Constant 2              97.677683                   96.076119   
9   Constant 6              98.034024                   95.479420   
10  Constant 3              97.480553                   95.636564   
11     Steps 1              96.977418                   95.233143   
12     Steps 2              96.911782                   95.055663   
13    Linear 1              97.025536                   94.784974   
14     Steps 4              96.481600                   94.406100   
15    Cosine 1              96.839493                   93.980950   
16     Steps 3              96.352323                   93.956158   

                               \
   Overall Score (Average), %   
0                   98.258848   
1                   97.343208   
2                   97.329239   
3                   97.187184   
4                   97.157092   
5                   97.036573   
6                   96.962431   
7                   96.886652   
8                   96.876901   
9                   96.756722   
10                  96.558559   
11                  96.105281   
12                  95.983722   
13                  95.905255   
14                  95.443850   
15                  95.410222   
16                  95.154240   

                                                        \
   Error rate (%) of column-wise density estimation, %   
0                                             0.949726   
1                                             2.010546   
2                                             1.920962   
3                                             2.316108   
4                                             2.350700   
5                                             2.226302   
6                                             2.250251   
7                                             2.333183   
8                                             2.322317   
9                                             1.965976   
10                                            2.519447   
11                                            3.022582   
12                                            3.088218   
13                                            2.974464   
14                                            3.518400   
15                                            3.160507   
16                                            3.647677   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
0                                                 0.380285   
1                                                 1.963639   
2                                                 1.507594   
3                                                 2.227463   
4                                                 2.010178   
5                                                 2.130774   
6                                                 2.194606   
7                                                 2.052890   
8                                                 2.404306   
9                                                 1.218693   
10                                                2.580102   
11                                                2.853078   
12                                                3.072292   
13                                                3.16

In [11]:
df[df[('', 'Model name')].isin(['TabDDPM', 'Constant 2', 'Constant 4', 'Cosine 3'])]

similarity                              \
   Model name Column Shapes Score, % Column Pair Trends Score, %   
0     TabDDPM              99.050274                   97.467421   
4    Cosine 3              97.649300                   96.664884   
6  Constant 4              97.749749                   96.175113   
8  Constant 2              97.677683                   96.076119   

                              \
  Overall Score (Average), %   
0                  98.258848   
4                  97.157092   
6                  96.962431   
8                  96.876901   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                            0.949726   
4                                            2.350700   
6                                            2.250251   
8                                            2.322317   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                0.380285   
4                                                2.010178   
6                                                2.194606   
8                                                2.404306   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                2.532579   
4                                                3.335116   
6                                                3.824887   
8                                                3.923881   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                    1.930225   
4                                                    2.144659   
6                                                    3.053694   
8                                                    3.393591   

                                    mle  \
  RMSE обучения на синтетических данных   
0                              0.634910   
4                              0.556092   
6                              0.547839   
8                              0.542569   

                                             detection       DCR  \
  RMSE обучения на синтетических данных, std     Score     Score   
0                                   0.044032  0.977026  0.507171   
4                                   0.043785  0.900264  0.505921   
6                                   0.044257  0.896107  0.511987   
8                                   0.049679  0.903409  0.505122   

          quality              
  alpha precision beta recall  
0        0.990344    0.541752  
4        0.963272    0.586571  
6        0.973713    0.558080  
8        0.969586    0.586073

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

\
              Model                              Type     Data   
0           TabDDPM                           tabddpm  beijing   
1   tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  beijing   
2   tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  beijing   
3   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  beijing   
4   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  beijing   
5   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  beijing   
6   tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  beijing   
7   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  beijing   
8   tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  beijing   
9   tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  beijing   
10  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  beijing   
11  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  beijing   
12  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  beijing   
13  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  beijing   
14  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  beijing   
15  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  beijing   
16  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  beijing   

               similarity                              \
   Column Shapes Score, % Column Pair Trends Score, %   
0               99.050274                   97.467421   
1               97.989454                   96.696962   
2               98.079038                   96.579441   
3               97.683892                   96.690476   
4               97.649300                   96.664884   
5               97.773698                   96.299449   
6               97.749749                   96.175113   
7               97.666817                   96.106487   
8               97.677683                   96.076119   
9               98.034024                   95.479420   
10              97.480553                   95.636564   
11              96.977418                   95.233143   
12              96.911782                   95.055663   
13              97.025536                   94.784974   
14              96.481600                   94.406100   
15              96.839493                   93.980950   
16              96.352323                   93.956158   

                               \
   Overall Score (Average), %   
0                   98.258848   
1                   97.343208   
2                   97.329239   
3                   97.187184   
4                   97.157092   
5                   97.036573   
6                   96.962431   
7                   96.886652   
8                   96.876901   
9                   96.756722   
10                  96.558559   
11                  96.105281   
12                  95.983722   
13                  95.905255   
14                  95.443850   
15                  95.410222   
16                  95.154240   

                                                        \
   Error rate (%) of column-wise density estimation, %   
0                                             0.949726   
1                                             2.010546   
2                                             1.920962   
3                                             2.316108   
4                                             2.350700   
5                                             2.226302   
6                                             2.250251   
7                                             2.333183   
8                                             2.322317   
9                                             1.965976   
10                                            2.519447   
11                                            3.022582   
12                                            3.088218   
13                                            2.974464   
14                                            3.518400   
15                                            3.160507   
16                                            3.64767

In [13]:
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False
                   ).reset_index(drop=True).to_excel(f'./_{datename}_notebooks/Result_united.xlsx')